## 【線形回帰】新生児の体重

新生児の性別と妊娠期間の両方を組み込んだ線形回帰モデルを構成してみる。但し、このモデルでは男児・女児で同じ傾きを持つ線形回帰モデルを適用するものとする。

In [ ]:
import pymc as pm
import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [8, 6]

## Load & Check Data

In [ ]:
data = pd.read_csv('babies_weight.csv')

In [ ]:
data.head(10)

## Preprocess & Scale Data

In [ ]:
def standardize(x):

    return (x - np.mean(x)) / np.std(x)

In [ ]:
x = data['weeks'].values
y = data['weight'].values

gender = data['gender'].values

In [ ]:
x_scaled = standardize(x)
y_scaled = standardize(y)

## Define Model & Inference

In [ ]:
with pm.Model() as model:

    a = pm.Normal('a', mu=0, sigma=10)
    b = pm.Normal('b', mu=0, sigma=10, shape=2)

    mu = a * x_scaled + b[gender]

    sd = pm.HalfCauchy('sd', 5)

    obs = pm.Normal('obs', mu=mu, sigma=sd, observed=y_scaled)

    b_diff = pm.Deterministic('b_diff', b[1] - b[0])

In [ ]:
with model:

    trace = pm.sample(3000, return_inferencedata=False)
    idata = pm.to_inference_data(trace, log_likelihood=True)

In [ ]:
idata

In [ ]:
az.plot_trace(idata)
plt.gcf().subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
az.summary(idata)

In [ ]:
az.plot_posterior(idata, var_names=['b_diff'], ref_val=0);

## Compare Models

In [ ]:
idata_imported = az.from_netcdf('idata.nc')

In [ ]:
idata_dict = {'Slope is same':idata, 'Slope is different':idata_imported}

In [ ]:
df_waic = az.compare(idata_dict, ic='waic', scale='deviance')
df_waic

In [ ]:
az.plot_compare(df_waic, figsize=(8, 4));

In [ ]:
df_loo = az.compare(idata_dict, ic='loo', scale='deviance')
df_loo

In [ ]:
az.plot_compare(df_loo, figsize=(8, 4));